In [1]:
import pandas as pd
import numpy as np
import os
os.environ['MKL_NUM_THREADS'] = '1'
from collections import Counter
import itertools
import json
import scipy.sparse as sparse
import pickle
import torch
import misc.util as util
from torch import nn

import importlib
from torch.utils.data import DataLoader
from misc.loader import AEDataset
from misc.util import *
from tqdm.auto import tqdm
from eval.rec_eval import *
import neuralsort.pl as pl
from models.loss import neuPrecLoss
from misc.loader import RecDataset
import models
from models.loss import *

In [2]:
import pickle
with open("data/parsed/ml-1m-new", 'rb') as f:
    (tr_users, val_users, te_users, train_data, val_tr, val_te, te_tr, te_te) = pickle.load(f)
empty = csr_matrix(train_data.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'data/parsed/ml-1m-new'

In [11]:
torch.LongTensor([3,4,5]).to(torch.LongTensor([1,2,3]).device)


tensor([3, 4, 5])

In [4]:
n_users, n_items = train_data[tr_users].shape
ae_dataset = AEDataset(train_data[tr_users])
total_anneal_steps = 50000
anneal_cap = 0.2


In [5]:
!ls saved_models/ml-1m-new

dae  ttt  vae


In [6]:
best_dae = torch.load(os.path.join("saved_models", "ml-1m-new", "dae"))

In [7]:
best_dae

{'model': MultiDAE(
   (layers): ModuleList(
     (0): Linear(in_features=3075, out_features=200, bias=True)
     (1): Linear(in_features=200, out_features=3075, bias=True)
   )
   (drop): Dropout(p=0.2, inplace=False)
   (dd): Dropout(p=0.2, inplace=False)
 ),
 'epoch': 90,
 'bs': 500,
 'lr': 0.0005,
 'dim': [200],
 'lamb': 0,
 'best_map10': 0.13484332813681035,
 'annel_caps': 0,
 'dropout': 0.2}

In [8]:
vin = val_tr[val_users]
vo = val_te[val_users]

In [9]:
tin = te_tr[te_users]
to = te_te[te_users]

In [10]:
wrapper = models.ae.implicitWrapper(best_dae['model'].eval(), naive_sparse2tensor, vae=True)

In [11]:
ranking_metrics_at_k(wrapper, tin, to, K=10)

{'precision': 0.13159999999999955,
 'recall': 0.23530452263344312,
 'map': 0.13635227135298564,
 'ndcg': 0.22833346604982988}

In [12]:
ranking_metrics_at_k(wrapper, tin, to, K=5)

{'precision': 0.16819999999999974,
 'recall': 0.16054058426269446,
 'map': 0.1458738888888888,
 'ndcg': 0.2187494307757626}

In [13]:
del best_dae['model']
del wrapper

In [14]:
vin_dense = torch.FloatTensor(np.asarray(vin.todense())).cuda()
tin_dense = torch.FloatTensor(np.asarray(tin.todense())).cuda()

In [15]:
list(zip([1,2],[3,4]))

[(1, 3), (2, 4)]

In [16]:
batch_size = best_dae['bs']
from implicit.evaluation import train_test_split
trin, trte = train_test_split(train_data[tr_users], 0.8)


In [20]:
qed = []
import models.ae
importlib.reload(models.ae)
import neuralsort.neuralobjs
importlib.reload(models.ae)
use_vae=True
loader = DataLoader(ae_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
model = models.ae.MultiDAE(best_dae['dim'] + [n_items], dropout=best_dae['dropout'])
model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=best_dae['lr'])
lm = -1
update_count = 0
tr_t = {
    'l': [],
    ('precision', 5):[],
    ('precision', 10):[],
    ('map', 5): [],
    ('map', 10): [],
    ('ndcg', 5): [],
    ('ndcg', 10): [] ,
}
val_t = {
    'l': [],
    ('precision', 5):[],
    ('precision', 10):[],
    ('map', 5): [],
    ('map', 10): [],
    ('ndcg', 5): [],
    ('ndcg', 10): [] ,
}
te_t = {
    'l': [],
    ('precision', 5):[],
    ('precision', 10):[],
    ('map', 5): [],
    ('map', 10): [],
    ('ndcg', 5): [],
    ('ndcg', 10): [] ,
}
for epoch in (range(1, 1000 + 1)):
    model = model.train()
    model.training = True
    tr_losses = []
    # train for one epoch
    for uid, rowl in (loader):
        row = rowl.float().cuda()
        uid = uid.cuda() 
        loss = None 
        scores = model.forward(row)
        loss  = models.loss.MultinomialLoss(row, scores)    
        (loss.mean()).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
        tr_losses.append(loss.detach().unsqueeze(-1)) 
        optimizer.step()
    
    if (epoch % 20 == 0):
        model.eval()
        model.training=False
        tr_loss = float(torch.cat(tr_losses).mean().detach().cpu().numpy())
        vad_scores = model.forward(vin_dense)
        vad_loss = models.loss.MultinomialLoss(vin_dense, vad_scores) 
        vad_loss = float(vad_loss.mean().detach().cpu().numpy())
        te_scores = model.forward(tin_dense)
        te_loss = models.loss.MultinomialLoss(tin_dense, te_scores) 
        te_loss = float(te_loss.mean().detach().cpu().numpy())
        tr_t['l'].append(tr_loss)
        val_t['l'].append(vad_loss)
        te_t['l'].append(te_loss)
        trwrapper = nfwrapper(model, naive_sparse2tensor, vae=True)
        wrapper = models.ae.implicitWrapper(model, naive_sparse2tensor, vae=True)
        ret = (tr_loss, vad_loss)
        for store, q, a in  zip([tr_t, val_t, te_t], [trin, vin, tin], [trte, vo, to]):
            for topk in [5, 10]:
                scs = ranking_metrics_at_k(wrapper, q, a, K=topk, num_threads=4)
                for ty in ['map', 'precision', 'ndcg']:
                    store[(ty, topk)].append(scs[ty])
            print(store)
        qed.append(ret)
no_add = qed

{'l': [266.47760009765625], ('precision', 5): [0.12251788268955388], ('precision', 10): [0.09859799713876985], ('map', 5): [0.09313733905579374], ('map', 10): [0.08116030143843699], ('ndcg', 5): [0.14809270194404592], ('ndcg', 10): [0.15144745936818754]}
{'l': [218.30418395996094], ('precision', 5): [0.11420000000000065], ('precision', 10): [0.09259999999999985], ('map', 5): [0.08922472222222234], ('map', 10): [0.07706628795036544], ('ndcg', 5): [0.14093238983992318], ('ndcg', 10): [0.1446015150843632]}
{'l': [208.54608154296875], ('precision', 5): [0.10660000000000056], ('precision', 10): [0.08619999999999987], ('map', 5): [0.08809638888888888], ('map', 10): [0.08015849883471905], ('ndcg', 5): [0.13735552209908125], ('ndcg', 10): [0.14508022320768554]}
{'l': [266.47760009765625, 246.50723266601562], ('precision', 5): [0.12251788268955388, 0.17688125894134388], ('precision', 10): [0.09859799713876985, 0.14097281831187547], ('map', 5): [0.09313733905579374, 0.14400778890478438], ('map',

{'l': [218.30418395996094, 204.91421508789062, 199.212646484375, 193.37843322753906, 189.06204223632812, 186.6862335205078], ('precision', 5): [0.11420000000000065, 0.1582, 0.1677999999999997, 0.17399999999999968, 0.17239999999999972, 0.1719999999999997], ('precision', 10): [0.09259999999999985, 0.1255999999999996, 0.13619999999999927, 0.13579999999999925, 0.13729999999999934, 0.13449999999999934], ('map', 5): [0.08922472222222234, 0.12689833333333328, 0.1385149999999997, 0.1376319444444443, 0.13837055555555522, 0.14201333333333324], ('map', 10): [0.07706628795036544, 0.11168620685311163, 0.12415832939657342, 0.12406381535021417, 0.12618826278659614, 0.12743812074829944], ('ndcg', 5): [0.14093238983992318, 0.19459560491395408, 0.20818465529268837, 0.2111863690872358, 0.2113909592751366, 0.2137849645852891], ('ndcg', 10): [0.1446015150843632, 0.1996332903037881, 0.21787018414402395, 0.21922877263802387, 0.2214164951438692, 0.22012375024531008]}
{'l': [208.54608154296875, 195.26820373535

{'l': [266.47760009765625, 246.50723266601562, 235.68702697753906, 224.41403198242188, 215.17138671875, 207.98143005371094, 203.20301818847656, 199.80905151367188, 197.6573486328125], ('precision', 5): [0.12251788268955388, 0.17688125894134388, 0.20686695278970071, 0.24314735336194787, 0.26832618025751337, 0.283319027181691, 0.30838340486409577, 0.3163376251788306, 0.3003719599427789], ('precision', 10): [0.09859799713876985, 0.14097281831187547, 0.1642060085836931, 0.1931044349070131, 0.21456366237482455, 0.2239198855507904, 0.24517882689556833, 0.25396280400572574, 0.24020028612303598], ('map', 5): [0.09313733905579374, 0.14400778890478438, 0.17487887458273715, 0.20774845016690544, 0.2365178032109365, 0.25258647273883394, 0.27899197265935516, 0.28876084883166475, 0.27327602924813243], ('map', 10): [0.08116030143843699, 0.12819615030614565, 0.15625032408713962, 0.18911156284163658, 0.2173107509972665, 0.23251160802369963, 0.25934190143497643, 0.2705709690011751, 0.25402240292344314], 

{'l': [266.47760009765625, 246.50723266601562, 235.68702697753906, 224.41403198242188, 215.17138671875, 207.98143005371094, 203.20301818847656, 199.80905151367188, 197.6573486328125, 195.91172790527344, 194.7261962890625], ('precision', 5): [0.12251788268955388, 0.17688125894134388, 0.20686695278970071, 0.24314735336194787, 0.26832618025751337, 0.283319027181691, 0.30838340486409577, 0.3163376251788306, 0.3003719599427789, 0.33030042918455377, 0.3078111587982866], ('precision', 10): [0.09859799713876985, 0.14097281831187547, 0.1642060085836931, 0.1931044349070131, 0.21456366237482455, 0.2239198855507904, 0.24517882689556833, 0.25396280400572574, 0.24020028612303598, 0.2703862660944238, 0.250701001430618], ('map', 5): [0.09313733905579374, 0.14400778890478438, 0.17487887458273715, 0.20774845016690544, 0.2365178032109365, 0.25258647273883394, 0.27899197265935516, 0.28876084883166475, 0.27327602924813243, 0.30436321729454807, 0.28050023843586125], ('map', 10): [0.08116030143843699, 0.1281

{'l': [208.54608154296875, 195.26820373535156, 189.74362182617188, 184.24630737304688, 179.91351318359375, 177.36598205566406, 173.99160766601562, 172.22540283203125, 173.28018188476562, 169.75057983398438, 171.4620361328125, 168.5426025390625], ('precision', 5): [0.10660000000000056, 0.1498000000000002, 0.16379999999999992, 0.16519999999999999, 0.16779999999999998, 0.16399999999999995, 0.15920000000000004, 0.15919999999999995, 0.156, 0.15360000000000001, 0.15519999999999995, 0.1472000000000001], ('precision', 10): [0.08619999999999987, 0.11949999999999969, 0.13159999999999952, 0.1309999999999996, 0.1304999999999995, 0.1302999999999995, 0.12829999999999953, 0.12689999999999954, 0.12459999999999961, 0.1205999999999996, 0.11979999999999964, 0.11789999999999963], ('map', 5): [0.08809638888888888, 0.12453055555555553, 0.13944777777777775, 0.1404302777777777, 0.14455916666666654, 0.14319583333333322, 0.13750416666666657, 0.13540416666666655, 0.13371361111111094, 0.13132166666666645, 0.13131

...

{'l': [266.47760009765625, 246.50723266601562, 235.68702697753906, 224.41403198242188, 215.17138671875, 207.98143005371094, 203.20301818847656, 199.80905151367188, 197.6573486328125, 195.91172790527344, 194.7261962890625, 193.7866973876953, 193.2414093017578, 192.95889282226562, 192.3410186767578, 192.03482055664062, 191.70272827148438, 191.3964385986328, 191.38487243652344, 191.11880493164062, 190.98757934570312, 190.92210388183594, 190.91159057617188, 190.65451049804688, 190.5201873779297, 190.24514770507812, 190.45602416992188, 190.49900817871094, 190.00379943847656, 189.79335021972656, 189.9722900390625, 189.8499298095703, 189.85455322265625, 189.763671875, 189.489013671875, 189.66200256347656, 189.51687622070312, 189.65420532226562, 189.5166015625, 189.4480438232422, 189.09022521972656, 189.27218627929688, 189.3397216796875, 189.460205078125, 189.19674682617188, 189.02767944335938, 188.983154296875, 189.06874084472656], ('precision', 5): [0.12251788268955388, 0.17688125894134388, 

{'l': [208.54608154296875, 195.26820373535156, 189.74362182617188, 184.24630737304688, 179.91351318359375, 177.36598205566406, 173.99160766601562, 172.22540283203125, 173.28018188476562, 169.75057983398438, 171.4620361328125, 168.5426025390625, 171.86500549316406, 168.340576171875, 169.2648468017578, 171.28990173339844, 168.9985809326172, 167.7911376953125, 171.1858367919922, 169.57928466796875, 168.67337036132812, 168.61541748046875, 171.66127014160156, 169.92369079589844, 168.02432250976562, 167.78253173828125, 169.9746551513672, 171.0953826904297, 170.2178192138672, 169.8775177001953, 168.26963806152344, 168.21881103515625, 170.9568328857422, 171.24456787109375, 171.06646728515625, 167.53073120117188, 167.7041015625, 171.3690948486328, 171.09072875976562, 168.30551147460938, 167.8826904296875, 171.1828155517578, 169.64552307128906, 167.76084899902344, 169.85789489746094, 171.62379455566406, 171.3760986328125, 170.37327575683594], ('precision', 5): [0.10660000000000056, 0.14980000000

{'l': [218.30418395996094, 204.91421508789062, 199.212646484375, 193.37843322753906, 189.06204223632812, 186.6862335205078, 183.14248657226562, 181.41604614257812, 182.7710418701172, 178.90760803222656, 180.90069580078125, 177.69985961914062, 181.3511505126953, 177.53329467773438, 178.53265380859375, 180.66497802734375, 178.16551208496094, 176.87832641601562, 180.5286407470703, 178.732666015625, 177.6972198486328, 177.65757751464844, 180.91366577148438, 179.0800323486328, 177.07803344726562, 176.80613708496094, 179.1636962890625, 180.42324829101562, 179.40194702148438, 179.02381896972656, 177.2932891845703, 177.2108154296875, 180.13302612304688, 180.46299743652344, 180.3035430908203, 176.49713134765625, 176.71376037597656, 180.65113830566406, 180.27711486816406, 177.35569763183594, 176.85858154296875, 180.525390625, 178.93246459960938, 176.79415893554688, 178.9792022705078, 180.92282104492188, 180.71107482910156, 179.50131225585938, 178.38552856445312], ('precision', 5): [0.11420000000

{'l': [266.47760009765625, 246.50723266601562, 235.68702697753906, 224.41403198242188, 215.17138671875, 207.98143005371094, 203.20301818847656, 199.80905151367188, 197.6573486328125, 195.91172790527344, 194.7261962890625, 193.7866973876953, 193.2414093017578, 192.95889282226562, 192.3410186767578, 192.03482055664062, 191.70272827148438, 191.3964385986328, 191.38487243652344, 191.11880493164062, 190.98757934570312, 190.92210388183594, 190.91159057617188, 190.65451049804688, 190.5201873779297, 190.24514770507812, 190.45602416992188, 190.49900817871094, 190.00379943847656, 189.79335021972656, 189.9722900390625, 189.8499298095703, 189.85455322265625, 189.763671875, 189.489013671875, 189.66200256347656, 189.51687622070312, 189.65420532226562, 189.5166015625, 189.4480438232422, 189.09022521972656, 189.27218627929688, 189.3397216796875, 189.460205078125, 189.19674682617188, 189.02767944335938, 188.983154296875, 189.06874084472656, 189.03050231933594, 189.0164031982422], ('precision', 5): [0.1

{'l': [208.54608154296875, 195.26820373535156, 189.74362182617188, 184.24630737304688, 179.91351318359375, 177.36598205566406, 173.99160766601562, 172.22540283203125, 173.28018188476562, 169.75057983398438, 171.4620361328125, 168.5426025390625, 171.86500549316406, 168.340576171875, 169.2648468017578, 171.28990173339844, 168.9985809326172, 167.7911376953125, 171.1858367919922, 169.57928466796875, 168.67337036132812, 168.61541748046875, 171.66127014160156, 169.92369079589844, 168.02432250976562, 167.78253173828125, 169.9746551513672, 171.0953826904297, 170.2178192138672, 169.8775177001953, 168.26963806152344, 168.21881103515625, 170.9568328857422, 171.24456787109375, 171.06646728515625, 167.53073120117188, 167.7041015625, 171.3690948486328, 171.09072875976562, 168.30551147460938, 167.8826904296875, 171.1828155517578, 169.64552307128906, 167.76084899902344, 169.85789489746094, 171.62379455566406, 171.3760986328125, 170.37327575683594, 169.3494415283203, 168.0986785888672], ('precision', 5

In [13]:
qed = []
import models.ae
importlib.reload(models.ae)
import neuralsort.neuralobjs
importlib.reload(models.ae)
use_vae=True
loader = DataLoader(ae_dataset, batch_size=batch_size, shuffle=True, num_workers=6)
if use_vae:
    model = models.ae.MultiVAE([200] + [n_items], dropout=0.5)
else:
    model = models.ae.MultiDAE([200] + [n_items], dropout=0.5)
model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=5 * 1e-4)
sc = neuralsort.neuralobjs.SC()
lm = -1 
tr_tt = {
    5: [],
    10: []
}
val_tt = {
    5: [],
    10: [],
}
lolis = []
update_count = 0
for epoch in (range(1, 1000 + 1)):
    model = model.train()
    model.training = True
    tr_losses = []
    loss2s =[] 
    # train for one epoch
    for uid, rowl in (loader):
        row = rowl.float().cuda()
        uid = uid.cuda() 
        loss = None 
        if use_vae:
            scores, mu, logvar = model.forward(row)
            if total_anneal_steps > 0:
                anneal_cap = min(anneal_cap, 1. * update_count / total_anneal_steps)
            else:
                anneal = anneal_cap
            loss  = models.ae.loss_function(scores, row, mu, logvar)
            update_count += 1

        else:
            scores = model.forward(row)
            loss  = models.loss.MultinomialLoss(row, scores)

        loss2 = neuMapLoss(sc, scores, row, topk=300, k=5, tau=15.0, use_top=True)
        loss2s.append(loss2.unsqueeze(-1))
        (loss.mean() + 10.0 * loss2.mean()).backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
        tr_losses.append(loss.detach().unsqueeze(-1)) 
        optimizer.step()
    
    if (epoch % 10 == 0):
        tr_loss = torch.cat(tr_losses).mean()
        lo2 = torch.cat(loss2s).mean()
        lolis.append(lo2)
        _in = torch.from_numpy(vin_dense).float().cuda()
        vad_scores, mu, logvar = model.forward(_in)
        vad_loss = models.ae.loss_function(vad_scores, _in, mu, logvar)
        vad_loss.mean().detach().cpu().numpy()
        model.eval()
        model.training=False                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
        wrapper1 = models.ae.implicitWrapper(model, train_data, naive_sparse2tensor, vae=True)
        wrapper2 = models.ae.implicitWrapper(model, vin, naive_sparse2tensor, vae=True)

        
        ret = (tr_loss.detach().cpu().numpy(), vad_loss.mean().detach().cpu().numpy())
        
        for topk in [5, 10]:
            tr_tt[topk].append(ranking_metrics_at_k(wrapper1, empty, train_data, K=topk, num_threads=4)['map'])
            val_tt[topk].append((ranking_metrics_at_k(wrapper2, vin, vo, K=topk, num_threads=4)['map']))
        print(["%0.4f" % x for x in ret], tr_tt[5][-1], tr_tt[10][-1],val_tt[5][-1], val_tt[10][-1])
        qed.append(ret)
add = qed

['294.7231', '225.8089'] 0.21826278174821165 0.17152883630953158 0.05616430555555556 0.05015200877110111
['274.4365', '219.9437'] 0.2750879604178133 0.21030542590868131 0.06868902777777773 0.06192627480158732
['280.2097', '216.5288'] 0.35857614073666827 0.2742169076872701 0.08582458333333337 0.07594075207860913
['275.0313', '214.3504'] 0.400465457210921 0.3105787457333225 0.09942861111111095 0.08769221812799184
['267.6081', '212.9479'] 0.43259849734286077 0.33781472371745475 0.10510222222222197 0.09244149911816571
['263.7416', '212.0777'] 0.4634359538207799 0.3622940053920485 0.1130180555555553 0.09898759511211869
['266.4737', '211.8793'] 0.4883085944658228 0.3832880855266988 0.11566236111111074 0.10196724600025156
['272.7971', '211.2510'] 0.509337548103352 0.401759022361108 0.11879930555555519 0.10581691649344904
['262.5125', '211.0949'] 0.5284835990470935 0.4176413090331327 0.12163013888888845 0.10819690641534364
['274.7474', '210.6121'] 0.5448414879970654 0.4326874071712478 0.122983

['259.1045', '214.6977'] 0.9268123511086737 0.8329643437506576 0.13897249999999972 0.12617133739606934
['265.1940', '214.8335'] 0.9272796408283005 0.8339701823529737 0.13918958333333298 0.12575000543272832
['263.5654', '214.5125'] 0.9297067986072995 0.8383749975414234 0.13796916666666656 0.1252059786785081
['258.7459', '214.9248'] 0.9311599780099016 0.8402338847446248 0.139772083333333 0.12628875015747018
['265.3243', '214.5748'] 0.931131574124983 0.8416007946965607 0.13900611111111094 0.12567849434681252
['257.1082', '214.8861'] 0.934390690855788 0.8463302887354356 0.13806486111111108 0.12572311799256713
['264.7551', '214.9235'] 0.9347544438336146 0.846072154675306 0.13944319444444422 0.1265741059618291
['264.0295', '215.1135'] 0.9377350192413472 0.8498813985615058 0.1396763888888886 0.12661996779730392
['264.2753', '214.9118'] 0.9366208539490622 0.84978738020823 0.1380765277777774 0.12572762117346908
['258.3012', '214.8421'] 0.9376378962800136 0.8499655549741576 0.1372198611111109 0.

In [16]:
lolis

[tensor(-0.4817, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-0.7677, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.0045, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.1744, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.2924, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.3773, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.4989, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.5715, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.5790, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.6430, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.7150, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.7087, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.7785, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.7870, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.8957, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.8911, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(-1.9152, device='cuda:0', grad_f